# Metrics of pre trained ResNeXt-101 and ResNet-101 with augmented images

In [1]:
import pandas as pd
import sys
import pickle

sys.path.insert(0, "./../../../src/utils/")

from utils import get_train_test_split

import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
from tqdm import tqdm

from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.gridspec as gridspec

from utils import get_scores

In [2]:
def get_averaged_df(score_dataframe):
    a = score_dataframe.groupby("model").agg(['mean', 'std'])
    a.columns = list(map('_'.join, a.columns.values))
    a["MP"] = round(a["MP_mean"], 2).astype(str) + " ± " + round(a["MP_std"], 2).astype(str)
    a["MMP"] = round(a["MMP_mean"], 2).astype(str) + " ± " + round(a["MMP_std"], 2).astype(str)
    a["mean_median_position"] = round(a["mean_median_position_mean"], 2).astype(str) + " ± " + round(a["mean_median_position_std"], 2).astype(str)
    a["MAP"] = round(a["MAP_mean"], 2).astype(str) + " ± " + round(a["MAP_std"], 2).astype(str)
    a["R@400"] = round(a["R@400_mean"], 2).astype(str) + " ± " + round(a["R@400_std"], 2).astype(str)
    a["R@200"] = round(a["R@200_mean"], 2).astype(str) + " ± " + round(a["R@200_std"], 2).astype(str)
    a["R@100"] = round(a["R@100_mean"], 2).astype(str) + " ± " + round(a["R@100_std"], 2).astype(str)
    a["R@50"] = round(a["R@50_mean"], 2).astype(str) + " ± " + round(a["R@50_std"], 2).astype(str)
    a["R@20"] = round(a["R@20_mean"], 2).astype(str) + " ± " + round(a["R@20_std"], 2).astype(str)

    b = a[["MP", "MMP", 'MAP','R@400', 'R@200', 'R@100', 'R@50', 'R@20']].reset_index()
    score_dataframe_avg_df = b.replace({"resnet": "ResNet-101", "resnext": "ResNeXT-101"})
    
    return score_dataframe_avg_df

In [3]:
data_folder = "./../../../data/"
annotations_by_hand = "./../../../data/manual_annotation/annotated_drawing_artwork_pairs.csv"
uid2path = "./../../../data/text_data/uid2path.pkl"

drawings_embeddings_path_resnext = "./../../../data/embeddings/drawings_learning_resnext-101_avg_280_epoch_0_initial.npy"
artworks_embeddings_path_resnext = "./../../../data/embeddings/famous_artworks_resnext-101_avg_280_epoch_0_initial.npy"

drawings_embeddings_path_resnet = "./../../../data/embeddings/drawings_learning_resnet101_avg_280_epoch_0_initial.npy"
artworks_embeddings_path_resnet = "./../../../data/embeddings/famous_artworks_resnet101_avg_280_epoch_0_initial.npy"

In [4]:
manually_annotated_df = pd.read_csv(annotations_by_hand)
with open(uid2path, "rb") as outfile:
    uid2path_df = pickle.load(outfile)
    
drawings_embeddings_resnext = np.load(drawings_embeddings_path_resnext, allow_pickle=True)
artworks_embeddings_resnext = np.load(artworks_embeddings_path_resnext, allow_pickle=True)

drawings_embeddings_resnet = np.load(drawings_embeddings_path_resnet, allow_pickle=True)
artworks_embeddings_resnet = np.load(artworks_embeddings_path_resnet, allow_pickle=True)

number_of_artworks = len(artworks_embeddings_resnext)

In [5]:
train_mets, val_mets,test_mets, non_train_mets = [], [], [], []

## Resnext

In [6]:
draw_art_pairs_df_full = pd.read_csv("./../../../data/model_learning/drawing_artwork_pair_train_test_split.csv")

draw_art_pairs_df = draw_art_pairs_df_full[
        (
            (draw_art_pairs_df_full["set"].isin(["train"]))
            | (
                (draw_art_pairs_df_full["set"].isin(["test", "val"]))
                & (
                    draw_art_pairs_df_full["drawing_id"]
                    == draw_art_pairs_df_full["parent"]
                )
            )
        )
    ].reset_index(drop=True)

train_mets.append(("0", "resnext") + get_scores(drawings_embeddings_resnext, artworks_embeddings_resnext, draw_art_pairs_df, ["train"], number_of_artworks))

val_mets.append(("0", "resnext") + get_scores(drawings_embeddings_resnext, artworks_embeddings_resnext, draw_art_pairs_df, ["val"], number_of_artworks))

test_mets.append(("0", "resnext") + get_scores(drawings_embeddings_resnext, artworks_embeddings_resnext, draw_art_pairs_df, ["test"], number_of_artworks))

non_train_mets.append(("v0", "resnext") + get_scores(drawings_embeddings_resnext, artworks_embeddings_resnext, draw_art_pairs_df, ["val", "test"], number_of_artworks))

100%|████████████████████████████████████████████████████████████████████████████████| 596/596 [01:00<00:00,  9.83it/s]


Mean position of artwork:  978.1517543859649
Mean of Minimum position of artwork:  1483.9874551971327
Mean of median position of artwork:  951.0439068100359
Mean Average Precision:  0.12250735646865588
recall @ 400 0.45698924731182794
recall @ 200 0.3888888888888889
recall @ 100 0.3172043010752688
recall @ 50 0.25985663082437277
recall @ 20 0.20430107526881722


100%|███████████████████████████████████████████████████████████████████████████████| 596/596 [00:02<00:00, 292.65it/s]


Mean position of artwork:  979.925
Mean of Minimum position of artwork:  1538.578947368421
Mean of median position of artwork:  972.7105263157895
Mean Average Precision:  0.169065186824115
recall @ 400 0.47368421052631576
recall @ 200 0.47368421052631576
recall @ 100 0.42105263157894735
recall @ 50 0.3684210526315789
recall @ 20 0.2631578947368421


100%|███████████████████████████████████████████████████████████████████████████████| 596/596 [00:02<00:00, 296.28it/s]


Mean position of artwork:  1018.2
Mean of Minimum position of artwork:  1675.157894736842
Mean of median position of artwork:  1056.4736842105262
Mean Average Precision:  0.09695262346483542
recall @ 400 0.42105263157894735
recall @ 200 0.3157894736842105
recall @ 100 0.3157894736842105
recall @ 50 0.3157894736842105
recall @ 20 0.21052631578947367


100%|███████████████████████████████████████████████████████████████████████████████| 596/596 [00:04<00:00, 121.16it/s]


Mean position of artwork:  999.0625
Mean of Minimum position of artwork:  1606.8684210526317
Mean of median position of artwork:  1014.5921052631579
Mean Average Precision:  0.13300890514447525
recall @ 400 0.4473684210526316
recall @ 200 0.39473684210526316
recall @ 100 0.3684210526315789
recall @ 50 0.34210526315789475
recall @ 20 0.23684210526315788


In [7]:
for i in range(1, 11):
    
    lbl = "v"+str(i)
    
    draw_art_pairs_df_full = pd.read_csv("./../../../data/model_learning/drawing_artwork_pair_train_test_split_"+lbl+".csv")
    
    draw_art_pairs_df = draw_art_pairs_df_full[
        (
            (draw_art_pairs_df_full["set"].isin(["train"]))
            | (
                (draw_art_pairs_df_full["set"].isin(["test", "val"]))
                & (
                    draw_art_pairs_df_full["drawing_id"]
                    == draw_art_pairs_df_full["parent"]
                )
            )
        )
    ].reset_index(drop=True)

    train_mets.append((str(i), "resnext") + get_scores(drawings_embeddings_resnext, artworks_embeddings_resnext, draw_art_pairs_df, ["train"], number_of_artworks))

    val_mets.append((str(i), "resnext") + get_scores(drawings_embeddings_resnext, artworks_embeddings_resnext, draw_art_pairs_df, ["val"], number_of_artworks))

    test_mets.append((str(i), "resnext") + get_scores(drawings_embeddings_resnext, artworks_embeddings_resnext, draw_art_pairs_df, ["test"], number_of_artworks))

    non_train_mets.append((str(i), "resnext") + get_scores(drawings_embeddings_resnext, artworks_embeddings_resnext, draw_art_pairs_df, ["val", "test"], number_of_artworks))

100%|████████████████████████████████████████████████████████████████████████████████| 531/531 [00:50<00:00, 10.42it/s]


Mean position of artwork:  1000.9831349206349
Mean of Minimum position of artwork:  1468.6270833333333
Mean of median position of artwork:  960.8552083333333
Mean Average Precision:  0.13977865462221348
recall @ 400 0.45625
recall @ 200 0.39791666666666664
recall @ 100 0.3458333333333333
recall @ 50 0.2833333333333333
recall @ 20 0.23541666666666666


100%|███████████████████████████████████████████████████████████████████████████████| 531/531 [00:03<00:00, 159.17it/s]


Mean position of artwork:  872.7333333333333
Mean of Minimum position of artwork:  1345.4666666666667
Mean of median position of artwork:  872.7333333333333
Mean Average Precision:  0.10518495133379899
recall @ 400 0.4
recall @ 200 0.4
recall @ 100 0.3
recall @ 50 0.26666666666666666
recall @ 20 0.2


100%|███████████████████████████████████████████████████████████████████████████████| 531/531 [00:02<00:00, 233.93it/s]


...

Mean position of artwork:  991.5208333333334
Mean of Minimum position of artwork:  1596.7727272727273
Mean of median position of artwork:  1017.6590909090909
Mean Average Precision:  0.1841059804972615
recall @ 400 0.36363636363636365
recall @ 200 0.2727272727272727
recall @ 100 0.22727272727272727
recall @ 50 0.22727272727272727
recall @ 20 0.18181818181818182


100%|███████████████████████████████████████████████████████████████████████████████| 596/596 [00:02<00:00, 297.15it/s]


Mean position of artwork:  1011.71875
Mean of Minimum position of artwork:  1623.4375
Mean of median position of artwork:  1011.71875
Mean Average Precision:  0.24101843073785956
recall @ 400 0.5
recall @ 200 0.375
recall @ 100 0.3125
recall @ 50 0.3125
recall @ 20 0.3125


100%|███████████████████████████████████████████████████████████████████████████████| 596/596 [00:03<00:00, 152.33it/s]


Mean position of artwork:  999.6
Mean of Minimum position of artwork:  1608.0
Mean of median position of artwork:  1015.1578947368421
Mean Average Precision:  0.20806911744067125
recall @ 400 0.42105263157894735
recall @ 200 0.3157894736842105
recall @ 100 0.2631578947368421
recall @ 50 0.2631578947368421
recall @ 20 0.23684210526315788


In [8]:
train_metrics_df = pd.DataFrame(train_mets, columns=["data_split_version", "model", "MP","MMP", "mean_median_position", 'MAP','R@400', 'R@200', 'R@100', 'R@50', 'R@20'])
val_metrics_df = pd.DataFrame(val_mets, columns=["data_split_version", "model", "MP","MMP", "mean_median_position", 'MAP','R@400', 'R@200', 'R@100', 'R@50', 'R@20'])
test_metrics_df = pd.DataFrame(test_mets, columns=["data_split_version", "model", "MP","MMP", "mean_median_position", 'MAP','R@400', 'R@200', 'R@100', 'R@50', 'R@20'])
non_train_metrics_df = pd.DataFrame(non_train_mets, columns=["data_split_version", "model", "MP","MMP", "mean_median_position", 'MAP','R@400', 'R@200', 'R@100', 'R@50', 'R@20'])

In [9]:
train_metrics_df["MAP"] = 100 * train_metrics_df["MAP"]
train_metrics_df["R@400"] = 100 * train_metrics_df["R@400"]
train_metrics_df["R@200"] = 100 * train_metrics_df["R@200"]
train_metrics_df["R@100"] = 100 * train_metrics_df["R@100"]
train_metrics_df["R@50"] = 100 * train_metrics_df["R@50"]
train_metrics_df["R@20"] = 100 * train_metrics_df["R@20"]

val_metrics_df["MAP"] = 100 * val_metrics_df["MAP"]
val_metrics_df["R@400"] = 100 * val_metrics_df["R@400"]
val_metrics_df["R@200"] = 100 * val_metrics_df["R@200"]
val_metrics_df["R@100"] = 100 * val_metrics_df["R@100"]
val_metrics_df["R@50"] = 100 * val_metrics_df["R@50"]
val_metrics_df["R@20"] = 100 * val_metrics_df["R@20"]

test_metrics_df["MAP"] = 100 * test_metrics_df["MAP"]
test_metrics_df["R@400"] = 100 * test_metrics_df["R@400"]
test_metrics_df["R@200"] = 100 * test_metrics_df["R@200"]
test_metrics_df["R@100"] = 100 * test_metrics_df["R@100"]
test_metrics_df["R@50"] = 100 * test_metrics_df["R@50"]
test_metrics_df["R@20"] = 100 * test_metrics_df["R@20"]

non_train_metrics_df["MAP"] = 100 * non_train_metrics_df["MAP"]
non_train_metrics_df["R@400"] = 100 * non_train_metrics_df["R@400"]
non_train_metrics_df["R@200"] = 100 * non_train_metrics_df["R@200"]
non_train_metrics_df["R@100"] = 100 * non_train_metrics_df["R@100"]
non_train_metrics_df["R@50"] = 100 * non_train_metrics_df["R@50"]
non_train_metrics_df["R@20"] = 100 * non_train_metrics_df["R@20"]

In [10]:
train_metrics_df

,version,model,MP,MMP,mean_median_position,MAP,R@400,R@200,R@100,R@50,R@20
0,v0,resnext,978.151754,1483.987455,951.043907,12.250736,45.698925,38.888889,31.720430,25.985663,20.430108
1,v1,resnext,1000.983135,1468.627083,960.855208,13.977865,45.625000,39.791667,34.583333,28.333333,23.541667
2,v2,resnext,982.285294,1442.441057,942.181911,13.619198,48.780488,43.089431,36.788618,30.691057,24.390244
3,v3,resnext,1038.881410,1636.300866,1029.542208,13.506510,48.268398,41.558442,35.281385,30.303030,24.458874
4,v4,resnext,1008.847390,1489.283951,963.528807,11.726500,47.530864,41.563786,34.567901,29.012346,21.193416
5,v5,resnext,1018.064922,1518.058233,979.737952,11.136251,45.582329,38.554217,32.128514,26.305221,19.879518
6,v6,resnext,964.398374,1524.037500,969.437500,14.045551,48.541667,41.666667,34.166667,29.583333,23.333333
7,v7,resnext,906.578947,1406.577061,909.670251,11.818437,47.849462,40.322581,33.512545,27.240143,21.326165
8,v8,resnext,901.218992,1353.024096,894.230924,13.833428,47.991968,41.767068,35.943775,29.718876,22.289157
9,v9,resnext,1019.092500,1528.578125,986.407118,10.942713,44.444444,38.194444,30.729167,25.173611,19.097222


In [11]:
val_metrics_df

,version,model,MP,MMP,mean_median_position,MAP,R@400,R@200,R@100,R@50,R@20
0,v0,resnext,979.925000,1538.578947,972.710526,16.906519,47.368421,47.368421,42.105263,36.842105,26.315789
1,v1,resnext,872.733333,1345.466667,872.733333,10.518495,40.000000,40.000000,30.000000,26.666667,20.000000
2,v2,resnext,1142.863636,1885.727273,1142.863636,9.332845,40.909091,22.727273,18.181818,9.090909,9.090909
3,v3,resnext,936.666667,1532.758621,957.758621,16.045766,37.931034,34.482759,31.034483,24.137931,20.689655
4,v4,resnext,870.725806,1356.166667,890.050000,12.996556,43.333333,33.333333,26.666667,23.333333,20.000000
5,v5,resnext,1169.086207,1938.172414,1169.086207,14.224647,37.931034,34.482759,27.586207,27.586207,17.241379
6,v6,resnext,814.425000,905.277778,658.916667,5.915792,44.444444,33.333333,27.777778,27.777778,22.222222
7,v7,resnext,1372.710526,2345.421053,1372.710526,8.441647,36.842105,36.842105,26.315789,26.315789,10.526316
8,v8,resnext,1276.258065,2013.666667,1208.433333,7.005585,36.666667,26.666667,13.333333,13.333333,6.666667
9,v9,resnext,1012.447368,1624.894737,1012.447368,17.666422,42.105263,26.315789,21.052632,21.052632,21.052632


In [12]:
test_metrics_df

,version,model,MP,MMP,mean_median_position,MAP,R@400,R@200,R@100,R@50,R@20
0,v0,resnext,1018.200000,1675.157895,1056.473684,9.695262,42.105263,31.578947,31.578947,31.578947,21.052632
1,v1,resnext,1170.238095,1940.476190,1170.238095,11.263226,52.380952,47.619048,28.571429,28.571429,19.047619
2,v2,resnext,939.214286,1499.851852,963.222222,10.681488,40.740741,37.037037,33.333333,33.333333,22.222222
3,v3,resnext,874.537037,1168.280000,800.420000,11.366725,48.000000,36.000000,36.000000,32.000000,16.000000
4,v4,resnext,1170.976190,2051.550000,1213.275000,24.361230,40.000000,35.000000,35.000000,30.000000,30.000000
5,v5,resnext,749.575000,1109.631579,773.710526,21.889611,42.105263,36.842105,36.842105,36.842105,31.578947
6,v6,resnext,1095.015152,1790.030303,1095.015152,13.501579,39.393939,33.333333,27.272727,27.272727,24.242424
7,v7,resnext,883.428571,1074.842105,743.368421,21.436893,47.368421,42.105263,31.578947,26.315789,21.052632
8,v8,resnext,924.444444,1448.888889,924.444444,26.590775,55.555556,44.444444,44.444444,44.444444,44.444444
9,v9,resnext,712.875000,1025.750000,712.875000,17.711640,62.500000,56.250000,50.000000,50.000000,37.500000


In [13]:
non_train_metrics_df

,version,model,MP,MMP,mean_median_position,MAP,R@400,R@200,R@100,R@50,R@20
0,v0,resnext,999.062500,1606.868421,1014.592105,13.300891,44.736842,39.473684,36.842105,34.210526,23.684211
1,v1,resnext,995.235294,1590.470588,995.235294,10.825149,45.098039,43.137255,29.411765,27.450980,19.607843
2,v2,resnext,1028.820000,1673.102041,1043.877551,10.075975,40.816327,30.612245,26.530612,22.448980,16.326531
3,v3,resnext,907.236842,1364.018519,884.916667,13.879543,42.592593,35.185185,33.333333,27.777778,18.518519
4,v4,resnext,991.980769,1634.320000,1019.340000,17.542426,42.000000,34.000000,30.000000,26.000000,24.000000
5,v5,resnext,997.857143,1610.208333,1012.583333,17.258695,39.583333,35.416667,31.250000,31.250000,22.916667
6,v6,resnext,989.132075,1477.764706,941.098039,10.824242,41.176471,33.333333,27.450980,27.450980,23.529412
7,v7,resnext,1115.837500,1710.131579,1058.039474,14.939270,42.105263,39.473684,28.947368,26.315789,15.789474
8,v8,resnext,1147.020408,1801.875000,1101.937500,14.350031,43.750000,33.333333,25.000000,25.000000,20.833333
9,v9,resnext,875.500000,1351.000000,875.500000,17.687093,51.428571,40.000000,34.285714,34.285714,28.571429


In [14]:
train_metrics_df.to_csv("./../../../data/model_learning/comparison/scores/baselines/train_scores_kfold.csv", index=False)
val_metrics_df.to_csv("./../../../data/model_learning/comparison/scores/baselines/val_scores_kfold.csv", index=False)
test_metrics_df.to_csv("./../../../data/model_learning/comparison/scores/baselines/test_scores_kfold.csv", index=False)
non_train_metrics_df.to_csv("./../../../data/model_learning/comparison/scores/baselines/non_train_scores_kfold.csv", index=False)

In [3]:
train_metrics_df= pd.read_csv("./../../../data/model_learning/comparison/scores/baselines/train_scores_kfold.csv")
val_metrics_df = pd.read_csv("./../../../data/model_learning/comparison/scores/baselines/val_scores_kfold.csv")
test_metrics_df = pd.read_csv("./../../../data/model_learning/comparison/scores/baselines/test_scores_kfold.csv")
non_train_metrics_df = pd.read_csv("./../../../data/model_learning/comparison/scores/baselines/non_train_scores_kfold.csv")

In [4]:
get_averaged_df(train_metrics_df)

,model,MP,MMP,MAP,R@400,R@200,R@100,R@50,R@20
0,ResNeXT-101,977.79 ± 46.2,1477.38 ± 77.21,12.54 ± 1.26,47.02 ± 1.46,40.52 ± 1.56,33.76 ± 1.93,28.05 ± 1.95,21.77 ± 1.95


In [5]:
get_averaged_df(val_metrics_df)

,model,MP,MMP,MAP,R@400,R@200,R@100,R@50,R@20
0,ResNeXT-101,1039.94 ± 178.51,1643.9 ± 389.54,12.5 ± 4.48,40.35 ± 3.63,32.98 ± 7.06,26.07 ± 7.51,23.53 ± 7.41,17.45 ± 6.1


In [6]:
get_averaged_df(test_metrics_df)

,model,MP,MMP,MAP,R@400,R@200,R@100,R@50,R@20
0,ResNeXT-101,959.11 ± 152.71,1491.63 ± 360.31,17.51 ± 6.39,47.29 ± 7.39,39.79 ± 7.26,35.08 ± 6.78,33.78 ± 7.35,27.13 ± 8.65


In [7]:
get_averaged_df(non_train_metrics_df)

,model,MP,MMP,MAP,R@400,R@200,R@100,R@50,R@20
0,ResNeXT-101,1004.3 ± 77.44,1584.34 ± 137.71,14.68 ± 3.39,43.22 ± 3.18,35.96 ± 3.99,29.94 ± 3.69,28.05 ± 3.73,21.59 ± 3.8
